<a href="https://colab.research.google.com/github/lcipolina/escher/blob/master/Lenstra/Luengo_Color.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install dipLib 

# to upload an image:
!python -m diplib download_bioformats

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.7 MB 12.3 MB/s 
Retrieving https://downloads.openmicroscopy.org/bio-formats/6.5.0/artifacts/bioformats_package.jar
[====================================================] 100%


In [8]:
# Bring Cris' image
!wget https://raw.githubusercontent.com/lcipolina/escher/master/Lenstra/PrintGallery.jpg --quiet

THINGS TO CHANGE IN HIS NEW CODE.

On the 'escher_forward', almost the last line:

add this:
 rot_angle = -1.1 # #use 0 this to get PG back (but straight images are not horizontal), use 0.65 to get straight images horizontal but don't get PG back #experimental
out = escher(src, src_scale, zoom, resolution, rot_angle) # rotation angle copied from esc


On "escher inverse"

change:
indx = (dip.Abs(Z.Real()) > limit) | (dip.Abs(Z.Imaginary()) > limit)

to:
indx = (dip.Abs(Z.Real()) > (limit * 0.998)) | (dip.Abs(Z.Imaginary()) > (limit * 0.998))

In [16]:
# FORWARD

import math
import diplib as dip


#***********************************************************************
# Grid handling functions
#***********************************************************************

def meshgrid(src, resolution=2, limit=1):
    '''
    Function to meshgrid
    :param src: source image
    :param resolution: scaling of output image w.r.t. input image
    :param limit: scaling of the mapping
    :return: meshgrid of X,Y coordinates as complex values
    '''
    out_sizes = [int(src.Size(0) * resolution), int(src.Size(1) * resolution)]
    out = dip.CreateCoordinates(out_sizes, {"frequency"}) * (2 * limit)
    out.MergeTensorToComplex()
    return out


def select_solution(Z, limit=1):
    '''
    Select, for each pixel (x,y), one of the solutions Z(:,x,y) that is inside the limit
    :param Z: mapped coordinates, as complex values
    :param limit: limit of the coordinate system
    '''
    # Set pixels out of bounds to (0,0)
    indx = (dip.Abs(Z.Real()) > limit) | (dip.Abs(Z.Imaginary()) > limit)
    Z[indx] = 0
    # For pixels where multiple solutions are OK, select the one furthest
    # away from the origin
    out = Z(0)
    for ii in range(1, Z.TensorElements()):
      tmp = Z(ii)
      indx = dip.Abs(tmp) > dip.Abs(out)
      if dip.Any(indx)[0][0]:
        out[indx] = tmp[indx]
    return out


def map_input(Z, src, limit=1):
    '''
    Function to create output image
    :param Z: mapped coordinates, as complex values
    :param src: source image
    :param limit: limit of the coordinate system
    '''
    # Convert complex values to coordinates
    Z.SplitComplexToTensor()
    # Center grid (this converts coordinates [-limit,limit] to [0,src.shape])
    Z += limit
    Z = dip.MultiplySampleWise(Z, [src.Size(0) / (2 * limit), src.Size(1) / (2 * limit)])
    # Generate output image
    out = dip.ResampleAt(src, Z, method="linear", fill=200)  # method="3-cubic" for better quality, but I don't think it matters
    out.SetColorSpace(src.ColorSpace())  # why is this necessary?
    return out


def escher_inverse(src, zoom=1, resolution=2, counter_rotation=0):
    '''
    Main function: applies the inverse Escher mapping
    :param src: image to apply the mapping to
    :param zoom: zooming of the ouput image
    :resolution: how many pixels compared to src
    :param counter_rotation: additional rotation given to the image to align visual axis
    '''
    # Create complex grid
    Z = meshgrid(src, resolution, zoom)
    # Apply Inverse Transformation
    Z = dip.Ln(Z)
    Z = dip.JoinChannels((Z + 2j * math.pi, Z, Z - 2j * math.pi, Z - 4j * math.pi)) # mutliple solutions to log(Z)
    Z /= (2j * math.pi - math.log(256)) / (2j * math.pi)
    Z = dip.Exp(Z)
    # Apply an additional rotation
    Z *= math.cos(counter_rotation) + 1j * math.sin(counter_rotation)
    # Select among the solutions for each pixel the best one
    Z = select_solution(Z, 1)
    # Create output picture
    return map_input(Z, src, 1)


#***********************************************************************
# Driver
#***********************************************************************

if __name__ == "__main__":
   # Parameters of the mapping
   resolution = 2                 # larger number means more output pixels

   # Image data to map
   src = dip.ImageRead('/content/fish-birds.jpg')
   # Picking the right cropping is important, as it determines the origin
   #src = src[104:741, 90:715]

   for zoom in range(9):
      # Generate mapped image
      out = escher_inverse(src, 2**(-zoom), resolution)
      # Write to file
      dip.ImageWrite(out, 'escher_straightened_'+str(zoom)+'.jpg')


In [17]:
import math
import diplib as dip
import numpy as np


#***********************************************************************
# Grid handling functions
#***********************************************************************

def meshgrid(src, resolution=2, limit=1):
    '''
    Function to meshgrid
    :param src: source image
    :param resolution: scaling of output image w.r.t. input image
    :param limit: scaling of the mapping
    :return: meshgrid of X,Y coordinates as complex values
    '''
    out_sizes = [int(src.Size(0) * resolution), int(src.Size(1) * resolution)]
    out = dip.CreateCoordinates(out_sizes, {"frequency"}) * (2 * limit)
    out.MergeTensorToComplex()
    return out


def select_solution(Z, limit=1):
    '''
    Select, for each pixel (x,y), one of the solutions Z(:,x,y) that is inside the limit
    :param Z: mapped coordinates, as complex values
    :param limit: limit of the coordinate system
    '''
    # Set pixels out of bounds to (0,0)
    indx = (dip.Abs(Z.Real()) > (limit * 0.998)) | (dip.Abs(Z.Imaginary()) > (limit * 0.998))
    Z[indx] = 0
    # For pixels where multiple solutions are OK, select the one furthest
    # away from the origin
    out = Z(0)
    for ii in range(1, Z.TensorElements()):
      tmp = Z(ii)
      indx = dip.Abs(tmp) > dip.Abs(out)
      if dip.Any(indx)[0][0]:
        out[indx] = tmp[indx]
    return out


def map_input(Z, src, src_scale, droste_scale):
    '''
    Function to create output image
    :param Z: mapped coordinates, as complex values
    :param src: images to apply the mapping to
    :param src_scale: scale for each of the images
    :param droste_scale: the scale at which we get back to the first image
    '''
    assert len(src) == len(src_scale)
    src_size = src[0].Sizes()
    for ii in range(1, len(src)):
        assert src[ii].Sizes() == src_size
    # Convert complex values to coordinates
    Z.SplitComplexToTensor()
    # Create a map for which pixels to take from which input image
    out = dip.Image(Z(0).Sizes(), src[0].TensorElements(), 'SFLOAT')
    out.SetColorSpace(src[0].ColorSpace())
    out.Fill(-1)
    for ii in reversed(range(len(src))):
        Z0 = dip.MultiplySampleWise(Z + (1 / src_scale[ii]), dip.Create0D([src_size[0] * src_scale[ii] / 2, src_size[1] * src_scale[ii] / 2]))
        src[ii].Convert('SFLOAT')
        tmp = dip.ResampleAt(src[ii], Z0, method="linear", fill=-1)  # method="3-cubic" for better quality, but I don't think it matters
        mask = (tmp(0) >= 0) & (out(0) < 0)
        if dip.Any(mask)[0][0]:
           out[mask] = tmp[mask]
    print("Number of pixels not assigned:", dip.Count(out(0) < 0))
    out.Convert("UINT8")
    return out


def escher(src, src_scale, zoom=1, resolution=10, counter_rotation=0):
    '''
    Main function: applies the Escher mapping
    :param src: images to apply the mapping to
    :param src_scale: scale for each of the images
    :param zoom: zooming of the ouput image, by default is 1
    :resolution: how many pixels compared to src
    :param counter_rotation: additional rotation given to the image to align visual axis
    '''
    # Create complex grid
    Z = meshgrid(src[0], resolution, zoom)
    # Apply Transformation
    Z = dip.Ln(Z)
    Z = dip.JoinChannels((Z + 2j * math.pi, Z, Z - 2j * math.pi, Z - 4j * math.pi)) # mutliple solutions to log(Z)
    Z *= (2j * math.pi - math.log(256)) / (2j * math.pi)
    Z = dip.Exp(Z)
    # Apply an additional rotation
    Z *= math.cos(counter_rotation) + 1j * math.sin(counter_rotation)
    # Select among the solutions for each pixel the best one
    Z = select_solution(Z, 1)
    # Create output picture
    return map_input(Z, src, src_scale, 256)


#***********************************************************************
# Driver
#***********************************************************************

if __name__ == "__main__":
   # Parameters of the mapping
   resolution = 1                # larger number means more output pixels
   zoom = 1                      # this value can be used to zoom in our out of the generated image

   # Image data to map
   src = []
   src_scale = []
   for ii in range(8):
      src.append(dip.ImageRead('escher_straightened_'+str(ii)+'.jpg'))
      src_scale.append(2**ii)     # doesn't match values used in escher_inverse.py
   
   # Generate mapped image
   out = escher(src, src_scale, zoom, resolution)
   
   # Write to file
   dip.ImageWrite(out, 'escher_composite.jpg')


Number of pixels not assigned: 0
